# ICESat-2 ATL14 Viewer
Interactive visualization and data extraction tool for ICESat-2 Gridded Land Ice Height data (ATL14)

In [ ]:
import ipywidgets
import IS2view
# autoreload
%load_ext autoreload
%autoreload 2

#### Set parameters for ATL14
- Asset: Location to get the data
- Directory: Working data directory
- Release: ATL14 data release (001, 002)
- Region: ATL14 data region (AA, CN, CS, GL, IS, RA, SV)
- Format: ATL15 data format to read (nc, zarr)

In [ ]:
IS2widgets = IS2view.widgets()
ipywidgets.VBox([
    IS2widgets.asset,
    IS2widgets.directory,
    IS2widgets.release,
    IS2widgets.region,
    IS2widgets.format,
])

### Interactive Mapping with Leaflet

Interactive maps within IS2view are built upon [ipyleaflet](https://ipyleaflet.readthedocs.io). Clicking and dragging will pan the field of view, and zooming will adjust the field of view. There are 2 polar stereographic projections available for mapping in IS2view ([North](https://epsg.io/3413) and [South](https://epsg.io/3031)). The map projection, map center and zoom level will all be set based on the ATL14 region selected. The available basemaps are NASA's Next Generation Blue Marble visualization for the Arctic and Antarctic regions.

Transects can be extracted by interactively drawing polylines on the leaflet map or programmatically using [geopandas GeoDataFrames](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html).

In [ ]:
m = IS2view.leaflet(IS2widgets.projection,
    center=IS2widgets.center,
    zoom=IS2widgets.zoom,
    draw_control=True,
    draw_tools='polyline',
    attribution=True)
m.map

### Query CMR for ATL14 granule
NASA's Common Metadata Repository (CMR) is a catalog of metadata records contained within the Earth Observing System Data and Information System (EOSDIS). ICESat-2 ATL14 data is archived at the NASA Distributed Active Archive Center (DAAC) at the [National Snow and Ice Data Center (NSIDC)](https://nsidc.org/daac/).

Currently, ATL14 data is only available from the on-premises NSIDC archives. NASA Earthdata is converting to data storage in the cloud using Amazon Web Services (AWS). IS2view will updated to allow data access through the NSIDC s3 bucket when publicly available.

In [ ]:
granule = IS2view.utilities.query_resources(
    asset=IS2widgets.asset.value,
    directory=IS2widgets.directory.value,
    product='ATL14',
    release=IS2widgets.release.value,
    region=IS2widgets.region.value,
    resolution='100m',
    format=IS2widgets.format.value
)

### Read and inspect ATL14 data
The ATL14 data will be read using [rioxarray](https://corteva.github.io/rioxarray/), an extension of [rasterio](https://rasterio.readthedocs.io/) and [xarray](https://xarray.dev/).

In [ ]:
variables = ['h', 'h_sigma', 'misfit_rms', 'misfit_scaled_rms']
ds = IS2view.io.from_file(granule, variable=variables,
    format=IS2widgets.format.value)
ds

#### Set plot parameters for ATL14

In [ ]:
IS2widgets.set_variables(ds)
IS2widgets.set_atl14_defaults()
ipywidgets.VBox([
    IS2widgets.variable,
    IS2widgets.cmap,
    IS2widgets.reverse,
])

### Add xarray dataset as an image service layer

In [ ]:
ds.leaflet.plot(m.map,
    vmin=IS2widgets.vmin, vmax=IS2widgets.vmax,
    variable=IS2widgets.variable.value,
    cmap=IS2widgets.colormap,
    opacity=0.75,
    enable_popups=True)

### Extract transects using selected geometries

In [ ]:
for feature in m.geometries['features']:
    ds.transect.plot(feature,
        variable=IS2widgets.variable.value,
    )

### Remove the image service layer from the map

In [ ]:
ds.leaflet.reset()